In [5]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
import accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

AttributeError: module 'bitsandbytes' has no attribute 'nn'

In [6]:
!python -m bitsandbytes
#had to install bitsandbytes-windows

C:\Users\rafsa\anaconda3\envs\weave\python.exe: No module named bitsandbytes.__main__; 'bitsandbytes' is a package and cannot be directly executed


In [2]:
qa_infile = "../res/prompt_db_target_drug_function_qa.txt"
df = pd.read_csv(qa_infile,sep="\t\t", index_col=None, header=None)#, columns=["question", "answer"])
df.columns = ["question", "answer"]
df

C:\Users\rafsa\AppData\Local\Temp\ipykernel_7496\2321868899.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(qa_infile,sep="\t\t", index_col=None, header=None)#, columns=["question", "answer"])


,question,answer
0,Q: Which drug is suitable for target Fibroblas...,A: Target Fibroblast growth factor receptor 1 ...
1,Q: Which drug is suitable for target Fibroblas...,A: Target Fibroblast growth factor receptor 1 ...
2,Q: Which drug is suitable for target Fibroblas...,A: Target Fibroblast growth factor receptor 1 ...
3,Q: Which drug is suitable for target Fibroblas...,A: Target Fibroblast growth factor receptor 1 ...
4,Q: Which drug is suitable for target Fibroblas...,A: Target Fibroblast growth factor receptor 1 ...
...,...,...
44426,Q: Which drug is suitable for target Food inta...,A: Target Food intake (FI) is druggable by HMR...
44427,Q: Which drug is suitable for target Histone s...,A: Target Histone synthesis (Histone synth) is...
44428,Q: Which drug is suitable for target Mitochond...,A: Target Mitochondrial gene transcription (MG...
44429,Q: Which drug is suitable for target P450-depe...,A: Target P450-dependent ergosterol synthesis ...


In [3]:
ds_raw = Dataset.from_pandas(df)
ds_raw

Dataset({
    features: ['question', 'answer'],
    num_rows: 44431
})

In [4]:
traintestvalid = ds_raw.train_test_split(test_size=0.2)
testvalid = traintestvalid["test"].train_test_split(test_size=0.5)
ds = DatasetDict({
    'train': traintestvalid['train'],
    'test': testvalid['test'],
    'valid': testvalid['train']})
ds


DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 35544
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 4444
    })
    valid: Dataset({
        features: ['question', 'answer'],
        num_rows: 4443
    })
})

In [6]:
device = "cuda" # the device to load the model onto

bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")


ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [ ]:
messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])